In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd gdrive/My Drive

[Errno 2] No such file or directory: 'gdrive/My Drive'
/content/gdrive/My Drive


In [ ]:
!ls final/train/train_new

0  1  2  3  train


In [ ]:
from keras.applications import InceptionV3
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
from skimage import io
from skimage.io import imread,imshow
from skimage import util
from skimage import data,filters
from keras.layers import Conv2D,BatchNormalization,Flatten,MaxPool2D,Dropout,Dense
from keras import layers,Model,models,optimizers,regularizers
from skimage.color import rgb2gray
from skimage.color import gray2rgb
from skimage.measure import compare_ssim
from skimage.transform import resize
import shutil
from keras.applications import Xception,InceptionV3,ResNet152V2,InceptionResNetV2,ResNet152V2,NASNetLarge,DenseNet201

In [ ]:
train_csv=pd.read_csv('train_data.csv')

In [ ]:
train_csv.set_index('Name',inplace=True)

In [ ]:
train_csv.value_counts('Type')

Type
3    2000
0    2000
2    1200
1     800
dtype: int64

In [ ]:
#分類 train
for i in train_csv.index:
  path='/content/gdrive/My Drive/final/train/train_new/train/'+i
  if train_csv.loc[i,'Type']== 0:
    shutil.move(path, '/content/gdrive/My Drive/final/train/train_new/0')
  elif train_csv.loc[i,'Type']== 1:
    shutil.move(path, '/content/gdrive/My Drive/final/train/train_new/1')
  elif train_csv.loc[i,'Type']== 2:
    shutil.move(path, '/content/gdrive/My Drive/final/train/train_new/2')
  elif train_csv.loc[i,'Type']== 3:
    shutil.move(path, '/content/gdrive/My Drive/final/train/train_new/3')

In [ ]:
#自己建立 CNN
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(324, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 109, 109, 64)      18496     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 54, 54, 64)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 186624)            0         
_________________________________________________________________
dense_8 (Dense)              (None, 324)               60466500  
_________________________________________________________________
dense_9 (Dense)              (None, 4)                

In [ ]:
#更改 數遽增幅參數
def image_data_generator(data_dir,data_augment=False,
                       batch_size=32,
                       target_size=(299, 299),
                       class_mode='categorical',
                       subset='training',
                       shuffle=True):
  if data_augment:
      datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=10,
																	 width_shift_range=0.1,
																	 height_shift_range=0.1,
                                   zoom_range=0.11,
																	 shear_range=0.12,
                                   validation_split=0.15,
                                   vertical_flip=True,
                                   horizontal_flip=True)
  else:
      datagen = ImageDataGenerator(rescale=1./255,validation_split=0.15)

  generator = datagen.flow_from_directory(data_dir,
                                          target_size=target_size,
                                          batch_size=batch_size,
                                          shuffle=shuffle,
                                          
                                          class_mode=class_mode,
                                          subset=subset)
  return generator

In [ ]:
train_generator = image_data_generator( "/content/gdrive/My Drive/final/train/train",data_augment=True)

Found 4798 images belonging to 4 classes.


In [ ]:
validation_generator = image_data_generator( "/content/gdrive/My Drive/final/train/train",data_augment=False,subset='validation')

Found 1198 images belonging to 4 classes.


In [ ]:
train_generator.class_indices

{'0': 0, '1': 1, '2': 2, '3': 3}

In [ ]:
#pre trained model Xception , input shape:(299,299,3) , 一層Dropout(0.5)防止過擬合,output用 sortmax activation
from keras.applications.xception import Xception
conv_base = Xception(weights='imagenet', include_top=False, input_shape=(299, 299, 3)) 
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

In [ ]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 10, 10, 2048)      20861480  
_________________________________________________________________
flatten_3 (Flatten)          (None, 204800)            0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 204800)            0         
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 819204    
Total params: 21,680,684
Trainable params: 21,626,156
Non-trainable params: 54,528
_________________________________________________________________


In [ ]:
train_generator = image_data_generator( "/content/gdrive/My Drive/final/train/train",data_augment=True)

Found 4798 images belonging to 4 classes.


In [ ]:
validation_generator = image_data_generator( "/content/gdrive/My Drive/final/train/train",data_augment=False,subset='validation')

Found 1198 images belonging to 4 classes.


In [ ]:
#pb 0.78 model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=50,
      epochs=30,
      validation_data=validation_generator)

In [ ]:
#儲存 Xception val 0.84 pb 0.78
model.save('model/final_ception_1.h5')

In [ ]:
retrain_datagen = ImageDataGenerator(rescale=1./255)
generator = retrain_datagen.flow_from_directory(
        train_dir,
        class_mode=None,
        target_size=(299,299),
        batch_size=1,
        shuffle=False)

Found 5996 images belonging to 4 classes.


In [ ]:
generator.class_indices

{'0': 0, '1': 1, '2': 2, '3': 3}

In [ ]:
pred=model.predict_generator(generator,verbose=1)

Instructions for updating:
Please use Model.predict, which supports generators.
5996/5996 [==============================] - 1133s 189ms/step


In [ ]:
predict=np.argmax(pred,axis=1)

In [ ]:
# 用 model 預測 train 檔 機率大於0.8移動Label
index=0
c=0
for name,label in zip(generator.filenames,predict):
  real_label=int(name[0])
  if real_label != label and pred[index][label]>0.8:
    c+=1
    print(name)
    print('real',real_label)
    print('pred',label)
    print(pred[index])
    print('-----------------')
    shutil.move("/content/gdrive/My Drive/final/train/train/"+name,'/content/gdrive/My Drive/final/train/train/'+str(label)+'/'+name[2:])
  index+=1

In [ ]:
train_generator = image_data_generator( "/content/gdrive/My Drive/final/train/train",data_augment=True)

Found 9349 images belonging to 4 classes.


In [ ]:
validation_generator = image_data_generator( "/content/gdrive/My Drive/final/train/train",data_augment=False,subset='validation')

Found 1646 images belonging to 4 classes.


In [ ]:
# 用新資料重新訓練 model
conv_base = Xception(weights='imagenet', include_top=False, input_shape=(299, 299,3) )
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

83689472/83683744 [==============================] - 1s 0us/step


In [ ]:
# pb 0.79
history = model.fit_generator(
      train_generator,
      steps_per_epoch=50,
      epochs=1,
      validation_data=validation_generator)

50/50 [==============================] - 80s 2s/step - loss: 1.0919 - acc: 0.8963 - val_loss: 1.1055 - val_acc: 0.8822


In [ ]:
conv_base = Xception(weights='imagenet', include_top=False, input_shape=(299, 299,3) )
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dropout(0.4))
model.add(layers.Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=50,
      epochs=35,
      validation_data=validation_generator)

In [ ]:
#預測test檔案
submit_dir="/content/gdrive/My Drive/final/test_data"
submit_datagen = ImageDataGenerator(rescale=1./255)
generator = submit_datagen.flow_from_directory(
        submit_dir,
        class_mode=None,
        classes=['no'],
        target_size=(299,299),
        batch_size=1,
        shuffle=False)

Found 2000 images belonging to 1 classes.


In [ ]:
pred=model.predict_generator(generator,verbose=1)
predict=np.argmax(pred,axis=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


2000/2000 [==============================] - 571s 285ms/step


In [ ]:
predict

array([3, 1, 0, ..., 3, 3, 0])

In [ ]:
#輸出
sample=pd.read_csv('sample_output.csv')
names=sample['Name']
submit=pd.DataFrame({'Name':names,'Type':predict})
submit.set_index('Name',inplace=True)
submit.to_csv('submit.csv',index=True)

In [ ]:
model.save('model/final_ception_new.h5')

In [ ]:
pred_add=model_add.predict_generator(generator,verbose=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


2000/2000 [==============================] - 33s 16ms/step


In [ ]:
predict_mix=np.argmax(pred+pred_add,axis=1)

In [ ]:
sample=pd.read_csv('sample_output.csv')
names=sample['Name']
submit=pd.DataFrame({'Name':names,'Type':predict_mix})
submit.set_index('Name',inplace=True)
submit.to_csv('submit.csv',index=True)

In [ ]:
#開始自行抓取資料
header = {
    'if-none-match-': '55b03-634508b9798ba9f4e118b697a946c895',
    'User-Agent':'Googlebot',
}

In [ ]:
url="https://shopee.tw/search?facet=2561%2C2560%2C1533%2C2573%2C14278&keyword=%E5%86%AC%E5%AD%A3%E8%A1%A3%E6%9C%8D&noCorrection=true&page=0"
res=requests.get(url,headers=header)

In [ ]:
soup = BeautifulSoup(res.text, 'html.parser')
items = soup.find_all("img", class_="_1T9dHf V1Fpl5")

In [ ]:
for page in range(20):
  url="https://shopee.tw/search?facet=100%2C2561%2C1487%2C2560%2C1533%2C2573%2C14278&keyword=%E5%86%AC%E5%AD%A3%E8%A1%A3%E6%9C%8D&noCorrection=true&page="+str(page)
  res=requests.get(url,headers=header).text
  soup = BeautifulSoup(res, 'html.parser')
  items = soup.find_all("img", class_="_1T9dHf V1Fpl5")
  for index,img in enumerate(items):
    img_dir=img.get('src')
    img_con=requests.get(img_dir)
    with open('./final/train/r09521227/data/0/'+'add_'+str(page)+'_'+str(index)+".jpg", "wb") as file:  
      file.write(img_con.content)

In [ ]:
for page in range(42):
  url="https://shopee.tw/search?facet=100%2C1473%2C1475%2C2165%2C1611%2C1477%2C1521&keyword=%E5%A4%8F%E5%AD%A3%E8%A1%A3%E6%9C%8D&noCorrection=true&page="+str(page)
  res=requests.get(url,headers=header).text
  soup = BeautifulSoup(res, 'html.parser')
  items = soup.find_all("img", class_="_1T9dHf V1Fpl5")
  for index,img in enumerate(items):
    img_dir=img.get('src')
    img_con=requests.get(img_dir)
    with open('./final/train/r09521227/data/1/'+'add_'+str(page)+'_'+str(index)+".jpg", "wb") as file:  # 開啟資料夾及命名圖片檔
      file.write(img_con.content)

In [ ]:
for page in range(38):
  url="https://shopee.tw/search?facet=9301%2C100%2C1485%2C1477&keyword=%E8%A3%99%E5%AD%90&noCorrection=true&page="+str(page)
  res=requests.get(url,headers=header).text
  soup = BeautifulSoup(res, 'html.parser')
  items = soup.find_all("img", class_="_1T9dHf V1Fpl5")
  for index,img in enumerate(items):
    img_dir=img.get('src')
    img_con=requests.get(img_dir)
    with open('./final/train/r09521227/data/2/'+'add_'+str(page)+'_'+str(index)+".jpg", "wb") as file:  # 開啟資料夾及命名圖片檔
      file.write(img_con.content)

In [ ]:
for page in range(22):
  url="https://shopee.tw/search?facet=1539%2C1527%2C1481%2C7649&keyword=%E7%89%9B%E4%BB%94%E8%A4%B2&noCorrection=true&page="+str(page)
  res=requests.get(url,headers=header).text
  soup = BeautifulSoup(res, 'html.parser')
  items = soup.find_all("img", class_="_1T9dHf V1Fpl5")
  for index,img in enumerate(items):
    img_dir=img.get('src')
    img_con=requests.get(img_dir)
    with open('./final/train/r09521227/data/3/'+'add_'+str(page)+'_'+str(index)+".jpg", "wb") as file:  # 開啟資料夾及命名圖片檔
      file.write(img_con.content)

In [ ]:
print(len(os.listdir('/content/gdrive/My Drive/final/train/train_add/0')))
print(len(os.listdir('/content/gdrive/My Drive/final/train/train_add/1')))
print(len(os.listdir('/content/gdrive/My Drive/final/train/train_add/2')))
print(len(os.listdir('/content/gdrive/My Drive/final/train/train_add/3')))

In [ ]:
train_generator = image_data_generator( "/content/gdrive/My Drive/final/train/r09521227/data",data_augment=True)

Found 10007 images belonging to 4 classes.


In [ ]:
validation_generator = image_data_generator( "/content/gdrive/My Drive/final/train/r09521227/data",data_augment=False,subset='validation')

Found 1764 images belonging to 4 classes.


In [ ]:
#再次訓練
conv_base = Xception(weights='imagenet', include_top=False, input_shape=(299, 299,3) )
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

In [ ]:
#使用另一pre trained model訓練
conv_base = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299,3) )
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dropout(0.2))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=50,
      epochs=35,
      validation_data=validation_generator)

In [ ]:
pred=model.predict_generator(generator,verbose=1)
predict=np.argmax(pred,axis=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


2000/2000 [==============================] - 611s 305ms/step


In [ ]:
sample=pd.read_csv('sample_output.csv')
names=sample['Name']
submit=pd.DataFrame({'Name':names,'Type':predict})
submit.set_index('Name',inplace=True)
submit.to_csv('submit.csv',index=True)

In [ ]:
for name in os.listdir('/content/gdrive/My Drive/final/train/r09521227/3_copy'):
  shutil.move(os.path.join('/content/gdrive/My Drive/final/train/r09521227/3_copy',name),'/content/gdrive/My Drive/final/train/train_add/3')
